In [1]:
import json
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from pathlib import Path

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
dataset_dir = r'..\dataset\npz'
test_public_images = np.load(Path(dataset_dir, 'test_public_cropped_480x480.npz'))['test_images']
test_private_images = np.load(Path(dataset_dir, 'test_private_cropped_480x480.npz'))['test_images']
test_images = np.concatenate((test_public_images, test_private_images))
print(f'test_images: {test_images.shape}')

public_json_file = r'..\dataset\test_public.json'
with open(public_json_file, 'r') as f:
    test_public_images_lst = json.load(f)['test']
private_json_file = r'..\dataset\test_private.json'
with open(private_json_file, 'r') as f:
    test_private_images_lst = json.load(f)['test']
test_images_lst = test_public_images_lst+test_private_images_lst
print(f'test_images_lst: {len(test_images_lst)}')

cls_file = r'..\dataset\classes.txt'
with open(cls_file, 'r') as f:
    classes = [line.rstrip() for line in f.readlines()]
print(f'classes: {len(classes)}')

test_images: (22308, 480, 480, 3)
test_images_lst: 22308
classes: 33


In [3]:
normalize = {'ImageNet': transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225]),
             'TestData': transforms.Normalize(mean=[0.427, 0.458, 0.374],
                                              std=[0.224, 0.221, 0.247])}
transform = nn.Sequential(
    normalize['ImageNet']
)

class MyDataset(Dataset):
    def __init__(self, x):
        self.image = x
        
    def __getitem__(self, index): 
        image = np.transpose(self.image[index], (2, 0, 1))
        image = torch.tensor(image, dtype=torch.float32).div_(255).cuda()
        image = transform(image)
        
        return image
    
    def __len__(self):
        return len(self.image)

In [4]:
batch_size = 64
test_data = DataLoader(MyDataset(test_images), batch_size=batch_size)

In [5]:
model_name = 'efficientnetv2_s_20221213_132051-last'
model_path = Path('models', model_name+'.pth')
model = torch.load(model_path).cuda()
#print(model)

In [6]:
preds_lst = []

model.eval()
with torch.no_grad():
    for batch in tqdm(test_data, unit='batch', desc='test: '):
        images = batch

        outputs = model(images)
        _, preds = torch.max(outputs.data, 1)

        preds_lst.append(preds.cpu().tolist())

pred_lst = [pred for preds in preds_lst for pred in preds]

test: 100%|███████████████████████████████████████████████████████████████████████| 349/349 [01:43<00:00,  3.36batch/s]


In [7]:
data = {'filename': test_images_lst, 'label': [classes[pred] for pred in pred_lst]}
df = pd.DataFrame(data)

csv_dir = 'test_results'
csv_file = Path(csv_dir, model_name+'-pub_pri.csv')
df.to_csv(csv_file, index=False, encoding='utf-8')